### Assignment-01

## 基于模式匹配的对话机器人实现

In [6]:
import random
import jieba
import re 
#判断是否以问号开头，是否均为字符（不含空格，数字）--杨松
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [4]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [10]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [8]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [16]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [11]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    for k,v in rules.items():
        if pat_match(k.split(),saying.split()):
            print (' '.join(subsitite(random.choice(v).split(), pat_to_dict(pat_match(k.split(),saying.split())))))
get_response('I need iPhone',response_pair)
        
    

Why do you neeed iPhone


In [273]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [274]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [12]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token:  
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)
        

In [2]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [264]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

### 现在是你的时间了

In [37]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [277]:
def get_response(saying, response_rules):
    for k,v in response_rules.items():
        if set(k.split()).intersection(set(saying.split())):
            print (' '.join(subsitite(random.choice(v).split(), pat_to_dict(pat_match_with_seg(k.split(), saying.split()))))) 
get_response('I was five',rules)
get_response('hello doctor wang', rules)

I already knew you were five .
Hi, how do you do?


### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [3]:
#encoding=utf-8
"""
以下为全部程序部分 .
目前有一个问题，例如以下语句：匹配'?*x I want ?*y'，我输入zhangsan I want apple 可以，但是输入'I want apple'则匹配失败。
"""
import random
import jieba
import re 
#将rule与saying未知部分一一对应，若rule中固定部分在saying中没有，返回False
def  filter_rules(rule,saying):
    dicts = {}
    if  ' ' in rule :
        #英文处理
        rule_list=rule.split(' ')
        saying_list=saying.split(' ')
        for i, el in enumerate(rule_list):
            if '?*' in el:
                el = el.replace('*', '')
                dicts[el] = saying_list[i]
            elif el in saying_list:
                pass
            else:return False
    else:
        #中文处理
        lists = []
        while '?*' in rule:
            i = rule.index('?*')
            lists.append(rule[i:i + 3])
            rule = rule.replace(rule[i:i + 3], '|')
        list_rule=rid_empty(rule.split('|'))
        for x in list_rule:
            if x in saying:
                saying=saying.replace(x,'|')
            else:return False
        list_saying=saying.split('|')
        for k,v in enumerate(lists):
                v = v.replace('*', '')
                dicts[v]=list_saying[k]
    return dicts

#过滤数组中空元素
def  rid_empty(strs):
    while '' in strs:
        strs.remove('')
    return strs

#输入问题获取回复
def  get_response(saying,rule_responses):
    for  k,v in rule_responses.items():
        rule_format=filter_rules(k,saying)
        if  not rule_format:continue
        response=random.choice(v)
        for  x,y in rule_format.items():
            if  x in response:
                response=response.replace(x,y)
        return response
print (get_response('张三他们是朋友吗？',rule_responses))
print (get_response('zhangsan I want Apple',rule_responses))
print (get_response('我是AI机器人',rule_responses))
print (get_response('生活中我学习你的优点',rule_responses))
print (get_response('你和我都聪明',rule_responses))
print (get_response(' ',rule_responses))

你觉得他们可能不是朋友？
Why do you want Apple
你为什么要提AI的事情？
其实很有可能我们互相学习
你觉得聪明有什么问题吗?
很有趣


### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [ ]:
"""以下改进版可进行持续聊天，有以下特点：
1.当用户输入短语符合指定rule规则但不全，机器人可引导补全。
2.当用户语句无法匹配到rule时，机器人可提示用户输入规则。
3.当用户语句包含再见，机器人作出再见回复，并自动退出程序。"""
import random
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    # '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
mid_reply=["什么？","啥？继续，我的朋友","？？？，然后呢？","嗯嗯，你接着说..."]
def  filter_rules(rule,saying):
    #以下处理是将rule中的未知量?*x,?*y等对应到saying未知，若不匹配返回错误
    dicts = {}
    if  ' ' in rule :
        #英文处理
        rule_list=rule.split(' ')
        saying_list=saying.split(' ')
        if  len(rule_list)!=len(saying):return  False
        for i, el in enumerate(rule_list):
            if '?*' in el:
                el = el.replace('*', '')
                dicts[el] = saying_list[i]
            elif el in saying_list:
                pass
            else:return False
    else:
        #中文处理
        lists = []
        while '?*' in rule:
            i = rule.index('?*')
            lists.append(rule[i:i + 3])
            rule = rule.replace(rule[i:i + 3], '|')
        list_rule=rid_empty(rule.split('|'))
        for x in list_rule:
            if x in saying:
                saying=saying.replace(x,'|')
            else:return False
        list_saying=saying.split('|')
        for k,v in enumerate(lists):
                v = v.replace('*', '')
                dicts[v]=list_saying[k]
    return dicts

def  rid_empty(strs):
    #去除数组中空字符串
    while '' in strs:
        strs.remove('')
    return strs

def  get_response(saying,rule_responses):
    #输入问题获取回复
        lists=list()
        for  k,v in rule_responses.items():
            lists.append(k.replace('?*',''))
            if  saying in k:
                continue_ask(saying)
            rule_format=filter_rules(k,saying)
            if  not rule_format:
                continue
            response=random.choice(v)
            for  x,y in rule_format.items():
                if  x in response:
                    response=response.replace(x,y)
            return response
        return  "还要麻烦你重新换一种方式说，我1岁智商，没听明白....,你可以用以下方式跟我交流：\n"+'|'.join(lists)+'\n或者说声再见我们下次聊....'
def  Interaction():
    #入口点，用户交互起始点，并通过它返回
    while True:
        saying=input("说说你的想法吧!\n===》")
        if  '再见' in saying:
            print ("再见啦，期待与你再次相见！！")
            return
        res=get_response(saying,rule_responses)
        print (res)

def continue_ask(saying):
    #若上一句符合要求，将进行进一步询问
    second_saying=input(random.choice(mid_reply)+'\n'+"===>")
    if   saying in second_saying:
        new_saying=second_saying
    elif ' ' in saying: #有空格表示输入的是英文
        new_saying=saying+' '+second_saying
    else:#表示输入中文
        new_saying=saying+second_saying
    return  get_response(new_saying,rule_responses)

Interaction()


### 问题4

**1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？**  
　　我的以上两个程序在设计模式以及语法结构上，我觉得存在一定的问题。希望老师能在此次批改作业的过程中帮我提出改进建议，并在以后的学习工作过程中我该如何改进避免。另外，如果程序出现bug还望老师能够批评指正，谢谢！  
  **2. 什么是数据驱动？数据驱动在这个程序里如何体现？**  
　　数据驱动是通过信息化手段及其他人工方式采集海量的数据，将数据进行组织形成信息，之后对相关的信息进行整合和提炼，在数据的基础上经过训练和拟合帮助使用者进行决策、改进的一种手段方式。  
　　在本程序中，若对用户提出的问题作出合理回复，需要大量的、具有代表性的数据支持。  
  **3. 数据驱动与 AI 的关系是什么？**
　　以下是我的个人理解:    
　　AI在实际生活生产应用中可以理解为一个个项目的概念，每一个实际使用的AI产品都是一个项目，可以说数据驱动在整个AI项目的周期中都具有核心作用。  
　　**在需求设计过程中**，数据驱动的作用是驱动建立项目目标。现在大家都提倡做什么事用数据说话，通过研究历史数据，我们总结出项目的痛点，而解决痛点就是我们的实际目标。同时，通过数据的表现形式、指标，我们找出对应的解决方案。  
　　**在项目建设过程中**，数据驱动形成产品。我们项目的每一步开发，都要结合实际使用情况，尤其是在产品的测试过程中，我们要通过大量数据来纠偏我们理论或者想象中的产品和实际差别。  
　　**在项目运营过程中**，数据驱动着完善产品，当产品面临实时的(相对)、最新的数据时，考验我们的产品能否适应时代的需求，这时候我们要对产品进行改进迭代，以至达到持续稳定运行的状态，这时候我们才算形成了真正的产品。再往后就是，站在巨人的肩膀上的创新了，那将是有一个项目周期。  
　　综上，我觉得数据驱动在所有的项目产品中都发挥着重要作用，在AI领域尤为明显。

一些参考 pattern

In [1]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
